In [1]:
import pandas as pd
import geopandas as gpd

data_path = '/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/'

In [2]:
#get BC regional district average incomes and get deciles
#from https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E

# BC Variables

In [3]:
BC_data = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/Canada/BC_income_v2.CSV', encoding='latin-1')
mob_vars = pd.read_csv(data_path + 'mobility_heat/mobility_heat_vars_clean_90_v1.csv')


In [4]:
lst = [x for x in BC_data['GEO_NAME'] if 'Regional district' in x]
BC_data = BC_data[BC_data['GEO_NAME'].isin(lst)]


household_income = BC_data[BC_data['CHARACTERISTIC_NAME'] == '  Median total income of household in 2020 ($)']
household_income['median_income'] = household_income['C1_COUNT_TOTAL']

population = BC_data[BC_data['CHARACTERISTIC_NAME'] == 'Population, 2021']
population['population'] = population['C1_COUNT_TOTAL']

age = BC_data[BC_data['CHARACTERISTIC_NAME'] == '  65 years and over']
age['65_and_over'] = age['C1_COUNT_TOTAL']

nonwhite = BC_data[BC_data['CHARACTERISTIC_NAME'] == '  Total visible minority population']
nonwhite['nonwhite_pop'] = nonwhite['C1_COUNT_TOTAL']



/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/243704373.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income['median_income'] = household_income['C1_COUNT_TOTAL']
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/243704373.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['population'] = population['C1_COUNT_TOTAL']
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/243704373.py:12: SettingWithCopyWarning: 
A value is trying to be set

In [5]:
ca_areas = pd.read_csv(data_path + 'census/Canada/BC_popdens.csv')
ca_areas = ca_areas.iloc[:, 1:]
#BC_data = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/Canada/BC_income_v2.CSV', encoding='latin-1')

us_areas = pd.read_csv(data_path + 'census/US/pop_dens.csv')




In [6]:

us_areas = us_areas[['NAME', 'State', 'B01001_calc_PopDensity']]

ca_areas.replace({'–': '-'}, regex=True, inplace=True)
ca_areas.replace('Metro Vancouver', 'Greater Vancouver', inplace=True)
ca_areas.replace('Stikine Region[a]', 'Stikine Region', inplace=True)


us_areas.columns = ['county', 'state', 'popdens']

ca_areas.columns = ['county', 'popdens']
ca_areas['state'] = 'British Columbia'

pop_dens = pd.concat([us_areas, ca_areas])

pop_dens['location'] = pop_dens['county'] + ', ' + pop_dens['state']

In [7]:
pop_dens

,county,state,popdens,location
0,Autauga County,Alabama,35.853419,"Autauga County, Alabama"
1,Baldwin County,Alabama,50.541504,"Baldwin County, Alabama"
2,Barbour County,Alabama,11.247981,"Barbour County, Alabama"
3,Bibb County,Alabama,13.973114,"Bibb County, Alabama"
4,Blount County,Alabama,34.515816,"Blount County, Alabama"
...,...,...,...,...
24,Squamish-Lillooet,British Columbia,2.600000,"Squamish-Lillooet, British Columbia"
25,Stikine Region,British Columbia,0.010000,"Stikine Region, British Columbia"
26,Strathcona,British Columbia,2.400000,"Strathcona, British Columbia"
27,Sunshine Coast,British Columbia,7.900000,"Sunshine Coast, British Columbia"


In [8]:
pop_dens = pop_dens[pop_dens['state'].isin(['British Columbia', 'California', 'Oregon', 'Washington'])]

In [9]:
pop_dens['popdens_dec'] = pop_dens.groupby('state')['popdens'].transform(lambda x: pd.qcut(x, 10, labels=False))
pop_dens['popdens_dec'] += 1

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/3147381589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_dens['popdens_dec'] = pop_dens.groupby('state')['popdens'].transform(lambda x: pd.qcut(x, 10, labels=False))
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/3147381589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_dens['popdens_dec'] += 1


In [10]:
age = age.groupby('DGUID').first().reset_index()

In [11]:
income_pop = household_income.merge(population[['DGUID', 'CHARACTERISTIC_NAME', 'population']], on='DGUID')

In [12]:
income_pop_age = income_pop.merge(age[['DGUID', 'CHARACTERISTIC_NAME', '65_and_over']], on='DGUID')

In [13]:
can_total = income_pop_age.merge(nonwhite[['DGUID', 'CHARACTERISTIC_NAME', 'nonwhite_pop']], on='DGUID')

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/2573391190.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'CHARACTERISTIC_NAME_x'} in the result is deprecated and will raise a MergeError in a future version.
  can_total = income_pop_age.merge(nonwhite[['DGUID', 'CHARACTERISTIC_NAME', 'nonwhite_pop']], on='DGUID')


In [14]:
can_total['district'] = [x.split(',')[0] for x in can_total['GEO_NAME']]

In [15]:
can_total['propor_over_65'] = can_total['65_and_over'] / can_total['population']

In [16]:
can_total['non_white_propor'] = can_total['nonwhite_pop'] / can_total['population']

In [17]:
can_total['income_dec'] = pd.qcut(can_total['median_income'],10,labels=False)
can_total['income_dec'] += 1

In [18]:
can_total['over65_dec'] = pd.qcut(can_total['propor_over_65'],10,labels=False)
can_total['over65_dec'] += 1

In [19]:
can_total['nonwhite_dec'] = pd.qcut(can_total['non_white_propor'],10,labels=False)
can_total['nonwhite_dec'] += 1

In [20]:
bc_mob_vars = mob_vars.merge(can_total[['district', 'income_dec',  'over65_dec', 'nonwhite_dec']], left_on='county', right_on='district')

In [21]:
bc_mob_vars.columns

Index(['Unnamed: 0.2', 'index', 'Unnamed: 0.1', 'Unnamed: 0',
       'country_region_code', 'country_region', 'state', 'county',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id',
       'date_heat', 'retail_heat', 'grocery_pharmacy_heat', 'parks_heat',
       'transit_heat', 'workplaces_heat', 'residential_heat', 'year',
       'location', 'control_day', 'temperature_2m_heat', 'heat_ID_x',
       'date_control', 'retail_control', 'grocery_pharmacy_control',
       'parks_control', 'transit_control', 'workplaces_control',
       'residential_control', 'retail_difference',
       'grocery_pharmacy_difference', 'parks_difference', 'transit_difference',
       'workplaces_difference', 'residential_difference', 'day_of_week',
       'weekend', 'holiday', 'us_holiday_heat', 'us_holiday', 'C', 'datetime',
       'temperature_2m_control', 'temp_diff', 'heat_ID_y', 'date',
       'Heat Wave Duration', 'dur_ranking', 'stage', 'stage_frac', 'district',
       'income_dec', '

# U.S. Variables

In [22]:
mob_vars = pd.read_csv(data_path + 'mobility_heat/mobility_heat_vars_clean_90_v1.csv')
#mob_vars.drop(['over65_dec', 'income_dec'],axis=1, inplace=True)


us_age = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/US/plus65.csv', encoding='latin-1')
us_income = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/US/county_income.csv', encoding='latin-1')
us_race = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/US/county_white.csv', encoding='latin-1')

In [23]:
total_pop = us_age[us_age['variable'] == 'total_pop']
total_pop['total_pop'] = total_pop['estimate']

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/3177610766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_pop['total_pop'] = total_pop['estimate']


In [24]:
us_age = us_age[us_age['variable'] != 'total_pop']

In [25]:
us_age = us_age.groupby('NAME').sum().reset_index()

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/1343126567.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  us_age = us_age.groupby('NAME').sum().reset_index()


In [26]:
us_age['total65plus'] = us_age['estimate']

In [27]:
us_age = us_age.merge(total_pop, on="NAME")

In [28]:
us_age['propor_over_65'] = us_age['total65plus'] / us_age['total_pop']

In [29]:
us_income['median_income'] = us_income['estimate']

In [30]:
us_income_age = us_age.merge(us_income, on='NAME')

In [31]:
us_income_age['state'] = [x.split(',')[-1].strip() for x in us_income_age['NAME']]

In [32]:
us_income_age = us_income_age[us_income_age['state'].isin(['California', 'Washington', 'Oregon'])]

In [33]:
us_race['white_pop'] = us_race['estimate']

In [34]:
us_race = us_race.merge(total_pop, on="NAME")

In [35]:
us_race['non_white_propor'] = 1 - (us_race['white_pop'] / us_race['total_pop'])

In [36]:
us_total = us_income_age.merge(us_race, on='NAME')

In [37]:
us_total['state'] = [x.split(',')[-1].strip() for x in us_total['NAME']]

In [38]:
us_total = us_total[us_total['state'].isin(['California', 'Washington', 'Oregon'])]

In [39]:
us_total = us_total[['NAME', 'propor_over_65', 'median_income', 'state', 'non_white_propor']]

In [40]:
us_total['income_dec'] = us_total.groupby('state')['median_income'].transform(lambda x: pd.qcut(x, 10, labels=False))
us_total['income_dec'] += 1

In [41]:
us_total['over65_dec'] = us_total.groupby('state')['propor_over_65'].transform(lambda x: pd.qcut(x, 10, labels=False))
us_total['over65_dec'] += 1

In [42]:
us_total['nonwhite_dec'] = us_total.groupby('state')['non_white_propor'].transform(lambda x: pd.qcut(x, 10, labels=False))
us_total['nonwhite_dec'] += 1

In [43]:
mob_vars = mob_vars.merge(us_total[['NAME', 'over65_dec', 'income_dec', 'nonwhite_dec']], left_on='location', right_on='NAME')

In [44]:
mob_vars['over65_dec'].min()

1

# Merge BC and U.S. and save

In [45]:
mob_vars = mob_vars.append(bc_mob_vars)

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_9487/2836113146.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mob_vars = mob_vars.append(bc_mob_vars)


In [46]:
mob_vars.drop('district', axis=1, inplace=True)

In [47]:
mob_vars = mob_vars.merge(pop_dens[['location', 'popdens_dec']], on='location')

In [48]:
mob_vars.to_csv(data_path + 'mobility_heat/mob_vars_final_clean_90_v1.csv')